In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import pywt
import os
import tqdm 
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
from numpy import arange, ones, pi
from scipy import cos, sin
from scipy.fftpack import fft, fftfreq, ifft
from scipy.stats import skew
import pywt
from scipy.stats import kurtosis
#cA = pywt.swt([1, 2, 3, 4, 5, 6], 'db1')

In [2]:
abspath = os.path.abspath("C:/Users/SIL/Desktop/sil/WORK/SKT AI FELLOWSHIP 3기/PHM 챌린지 데이터셋 조사/phm-ieee-2012-data-challenge-dataset-master/phm-ieee-2012-data-challenge-dataset-master/Raw Data")

In [3]:
# TOP FIVE POINTS
def five_points(data):
    y_hori = np.array(data['Horiz']).astype(float)
    y_verti = np.array(data['Verti']).astype(float)
    
    # Top five points mean
    hori = np.sort(y_hori, axis=0)[::-1][:5].mean()
    verti = np.sort(y_verti, axis=0)[::-1][:5].mean()

    return hori,verti

In [30]:
def dwt(data):
    eli_len = len(data) % 64
    data = data.iloc[:-eli_len]
        
    y_hori = data['Horiz'] 
    y_verti = data['Vert'] 
            
    h_coff = pywt.swt(list(y_hori), 'db4',level=5,axis=0)
    v_coff = pywt.swt(list(y_verti), 'db4',level=5,axis = 0)
    
    
    # concat wavelet coff
    h_data = pd.concat([pd.DataFrame(i).T for i in h_coff],axis =1)
    v_data = pd.concat([pd.DataFrame(i).T for i in v_coff],axis =1)
    
    # set col name
    col_name_h = ['h_cA5','h_cD5','h_cA4','h_cD4','h_cA3','h_cD3','h_cA2','h_cD2','h_cA1','h_cD1']
    col_name_v = ['v_cA5','v_cD5','v_cA4','v_cD4','v_cA3','v_cD3','v_cA2','v_cD2','v_cA1','v_cD1']

    h_data.columns = col_name_h
    v_data.columns = col_name_v
    
    # merge(기본, H coffs , V coffs)
    final = pd.concat([data,h_data,v_data],axis= 1)
    final_value = final.rolling(5).mean().max()
    return final_value

In [34]:
#getting # five highest points
def fix_file(df):
    df = pd.Series(df[df.columns[0]])
    #데이터 형식이 안맞는 경우 
    fixed = pd.DataFrame()
    fixed['H'] = df.str.split(';').str[0]
    fixed['M'] = df.str.split(';').str[1]
    fixed['S'] = df.str.split(';').str[2]
    fixed['MS'] = df.str.split(';').str[3]
    fixed['Horiz'] = df.str.split(';').str[4]
    fixed['Vert'] = df.str.split(';').str[5]
    return fixed


def read_data(path):
    file_list = os.listdir(path)
    total_list = []
    for i in tqdm.tqdm(range(len(file_list[:-1]))):
        
        tem = pd.read_csv(path+'/'+file_list[i])
        if len(tem.columns) == 6:
            tem.columns = ['H','M','S','MS','Horiz','Vert']
            
            # Changing function depending on purpose
            #five_point_results = five_points(tem)
            
            five_point_results = dwt(tem)
            total_list.append(five_point_results)
            
            
        if len(tem.columns) == 1:
            tem2 = fix_file(tem)
            tem2.columns = ['H','M','S','MS','Horiz','Vert']
            
            
            # Changing function depending on purpose
            #five_point_results = five_points(tem2)
            five_point_results = dwt(tem2)
            total_list.append(five_point_results)        
            
        else:
            continue
            
    return total_list 



files = os.listdir(abspath)
files


# TEST/LEARNING (파일명)
for get_file in files[2:-1]:
    path_dir = abspath+"/"+get_file
    file2 = os.listdir(path_dir)
    # Bearing file 접근
    for file in file2:
        path2 = path_dir+'/'+file
        tem_fft = read_data(path2)
        fft_data = pd.DataFrame(tem_fft)
        
        # CSV 저장        
        fft_data.to_csv(abspath+'/dwt/'+file+"_dwt.csv",index=False)

        # PLOT 결과 저장
        plt.title(file+"top5")
        plt.xlabel("Observation #")
        plt.ylabel("Feature (g)")
        print(fft_data.head())
        plt.plot(fft_data['Horiz'],label='Horizontal')
        plt.plot(fft_data['Vert'],label='Vertical')

        #plt.scatter(range(len(fft_data)),fft_data['hori'],cmap='Greens',edgecolors='red',linewidth=2,s=10,label='Horizontal')
        #plt.scatter(range(len(fft_data)),fft_data['verti'],cmap='Greens',edgecolors='blue',linewidth=2,s=10,label='Vertical')
        plt.legend(loc = 'upper right')
        plt.savefig(abspath+'/dwt/'+file+'_top5.png')        
        plt.close()
        print("================="+file+" DONE ================")
        
        #plt.show()
        #tem.to_csv('/'+ get_file+'/'+file+".csv",index= False)    


  0%|                                                                                         | 0/1664 [00:00<?, ?it/s]

     H     M     S        MS   Horiz    Vert     h_cA5     h_cD5     h_cA4  \
0  8.0  33.0   1.0  475584.0  1.1060  0.4304  1.173360  1.613077  1.549534   
1  8.0  33.0  11.0  475584.0  0.7696  0.3232  1.303490  1.406551  1.358121   
2  8.0  33.0  21.0  475584.0  0.9492  0.3980  1.098965  1.196626  1.356523   
3  8.0  33.0  31.0  475584.0  0.6804  0.4240  1.399907  1.542751  1.531483   
4  8.0  33.0  41.0  475584.0  0.8226  0.3318  1.170126  1.673959  1.414938   

      h_cD4  ...     v_cA5     v_cD5     v_cA4     v_cD4     v_cA3     v_cD3  \
0  2.822171  ...  0.796336  0.416683  0.766762  0.484787  0.785301  0.567621   
1  1.878046  ...  0.774313  0.415722  0.726580  0.503060  0.654779  0.655930   
2  1.842371  ...  0.668708  0.435307  0.626037  0.555270  0.533835  0.603687   
3  1.338737  ...  0.702171  0.453670  0.597321  0.454479  0.618643  0.621314   
4  1.884809  ...  0.542962  0.363837  0.494145  0.527419  0.484552  0.603774   

      v_cA2     v_cD2     v_cA1     v_cD1  
0  0.7

  0%|                                                                                         | 0/2872 [00:00<?, ?it/s]

     H    M     S        MS   Horiz    Vert     h_cA5     h_cD5     h_cA4  \
0  8.0  8.0   0.0  522460.0  0.6080  0.4874  0.795731  0.675280  0.737573   
1  8.0  8.0  10.0  522460.0  0.7300  0.4318  0.787722  0.852297  0.776171   
2  8.0  8.0  20.0  522460.0  0.5962  0.5580  0.997018  0.550837  0.820731   
3  8.0  8.0  30.0  522460.0  0.6996  0.4856  1.283305  1.214784  1.588035   
4  8.0  8.0  40.0  522460.0  0.5322  0.5116  1.278861  0.679043  1.147172   

      h_cD4  ...     v_cA5     v_cD5     v_cA4     v_cD4     v_cA3     v_cD3  \
0  1.187298  ...  0.471559  0.352723  0.441771  0.516367  0.605818  0.807877   
1  1.263581  ...  0.507723  0.278481  0.410693  0.477432  0.544140  0.849437   
2  1.039189  ...  0.322719  0.277308  0.264363  0.532349  0.355448  0.858072   
3  1.722120  ...  0.470807  0.291700  0.437580  0.599185  0.461942  1.156861   
4  1.347487  ...  0.435501  0.276721  0.364184  0.577051  0.577939  0.917107   

      v_cA2     v_cD2     v_cA1     v_cD1  
0  0.650030 

  0%|                                                                                         | 0/2855 [00:00<?, ?it/s]

...

     H     M     S        MS   Horiz    Vert     h_cA5     h_cD5     h_cA4  \
0  8.0  39.0  57.0  669336.0  0.9184  0.4350  1.849338  2.321660  2.631825   
1  8.0  40.0   7.0  669336.0  1.3956  0.3710  2.953996  3.282134  3.577657   
2  8.0  40.0  17.0  669336.0  1.0050  0.3902  1.950156  2.697391  2.738707   
3  8.0  40.0  27.0  669336.0  0.8336  0.3802  1.564616  1.073530  1.317981   
4  8.0  40.0  37.0  669336.0  0.8108  0.4534  1.743622  1.249418  1.870207   

      h_cD4  ...     v_cA5     v_cD5     v_cA4     v_cD4     v_cA3     v_cD3  \
0  3.603610  ...  0.621556  0.432373  0.533732  0.668565  0.796897  0.577157   
1  6.189089  ...  0.538103  0.354003  0.497272  0.579220  0.663764  0.623279   
2  4.955341  ...  0.518442  0.363053  0.428784  0.567270  0.619740  0.642229   
3  2.372240  ...  0.643799  0.368642  0.589556  0.588770  0.577096  0.484888   
4  1.455055  ...  0.542405  0.444588  0.485655  0.669306  0.535370  0.665917   

      v_cA2     v_cD2     v_cA1     v_cD1  
0  0.7

  0%|                                                                                         | 0/2696 [00:00<?, ?it/s]

     H    M     S        MS   Horiz    Vert     h_cA5     h_cD5     h_cA4  \
0  8.0  3.0  36.0  559960.0  0.7042  0.3102  0.984597  0.737990  0.921197   
1  8.0  3.0  46.0  559960.0  0.7208  0.2996  0.858875  0.826821  1.087336   
2  8.0  3.0  56.0  559960.0  0.8352  0.4306  0.887186  0.858857  0.905362   
3  8.0  4.0   6.0  559960.0  0.5478  0.3308  0.774826  0.815716  0.817696   
4  8.0  4.0  16.0  559960.0  0.6494  0.3788  0.966487  0.743532  0.764653   

      h_cD4  ...     v_cA5     v_cD5     v_cA4     v_cD4     v_cA3     v_cD3  \
0  1.526032  ...  0.466774  0.226050  0.392384  0.513805  0.479815  0.541525   
1  1.601476  ...  0.424013  0.205204  0.340039  0.472698  0.452269  0.569378   
2  1.736573  ...  0.496403  0.196901  0.436052  0.637544  0.534256  0.666104   
3  1.103955  ...  0.482698  0.248833  0.343705  0.602560  0.384917  0.595756   
4  1.809069  ...  0.433722  0.291122  0.369396  0.558061  0.500156  0.683521   

      v_cA2     v_cD2     v_cA1     v_cD1  
0  0.523588 

 86%|██████████████████████████████████████████████████████████████████▊           | 2311/2696 [30:43<05:07,  1.25it/s]


ValueError: Data must have non-zero size